### we will compute the correlation with between-group differences in cortical eigenvectors and neurotransmitter distributions

In [ ]:
import os
import matplotlib.pyplot as plt

from brainspace.datasets import load_group_fc, load_parcellation, load_conte69
from enigmatoolbox.permutation_testing import spin_test

from scipy import io

import numpy as np
import matplotlib.pylab as plt
import scipy.io
import re

import pandas as pd
from scipy.stats import pearsonr

In [ ]:
from statsmodels.stats.multitest import fdrcorrection
import sys
mod = sys.modules[__name__]

In [ ]:
mat_file_name = "./data/receptor.mat"
mat_file = scipy.io.loadmat(mat_file_name)

In [ ]:
receptor = mat_file['receptor']['data_schaefer200'][0][0]
ctx_tval = pd.read_csv('./fig1/total_t_value.csv',header=None)
receptor = receptor.T
ctx_tval = np.array((ctx_tval))
ctx_tval = ctx_tval.reshape(-1)

In [ ]:
rval = []
for i in range(0,18+1):
    rval.append(pearsonr(ctx_tval, receptor[:,i])[0])

In [ ]:
names = mat_file['receptor']['name'][0][0]

receptor_name = []
for i in range(len(names)):
    receptor_name.append(names[i][0][0])

### perform spin permutation test

In [ ]:
for trial in range(1,100+1):
    print('\033[1m' + f'{i}th trial' + '\033[0m')
    per_p_list = []
    for i in range(0,18+1):
        ctx_pet_p, ctx_pet_d = spin_test(ctx_tval,receptor[:,i], surface_name='fsa5', parcellation_name='schaefer_200',
                                         type='pearson', n_rot=1000, null_dist=True)
        per_p_list.append(ctx_pet_p)
        globals()['tval{}_{}'.format(trial,str(i+1))] = ctx_pet_d

    per_p_list = np.array((per_p_list))
    globals()['per_p_list{}'.format(trial)] = per_p_list

In [ ]:
for i in range(1,100+1):
    per_p_list = getattr(mod,'per_p_list{}'.format(i))
    fdr_perm_p = fdrcorrection(per_p_list)[1]
    globals()['fdr_perm_p{}'.format(i)] = fdr_perm_p

In [ ]:
receptor_name = np.array((receptor_name))

we get the values based on p-value < 0.055

In [ ]:
for i in range(1,100+1):
    fdr_perm_p = getattr(mod,'fdr_perm_p{}'.format(i))
    per_sig_idx = np.where(fdr_perm_p < 0.055)
    per_sig_val = receptor_name[per_sig_idx]
    globals()['per_sig_val{}'.format(i)] = per_sig_val

In [ ]:
for i in range(1,100+1):
    fdr_perm_p = getattr(mod,'fdr_perm_p{}'.format(i))
    dat = pd.DataFrame([x for x in zip(rval,fdr_perm_p)]).T
    dat.columns = [i for i in receptor_name]
    dat.index = ['r_val','p_val']
    dat = np.round(dat,2)
    globals()['dat{}'.format(i)] = dat

the results during the 100 trials are saved as csv files as follows

In [ ]:
# for i in range(1,100+1):
#     dat = getattr(mod, 'dat{}'.format(i))
#     dat.to_csv('./fig3/permutation_result/dat{}.csv'.format(i))

we will find which of the 100 trials has a significant value

In [ ]:
per_sig_all = np.array(())
for i in range(1,100+1):
    per_sig_val = getattr(mod, 'per_sig_val{}'.format(i))
    per_sig_all = np.concatenate([per_sig_all,per_sig_val])

In [ ]:
per_sig_all = np.unique(per_sig_all, return_counts=True)
sig_table = pd.DataFrame([x for x in zip(per_sig_all[0],per_sig_all[1])]).T
#     tmp.columns = [i for i in receptor_name]
#     tmp.index = ['r_val','p_val']
sig_table = np.round(sig_table,2)

In [ ]:
sig_table

> As a result of the above, it was found that the 5HT6, A4B2, CB1, H3, M1, MOR, NMDA, mGluR5 were significant under p-value < 0.05.

Determine the p-value that has the highest frequency out of 100 times as the probability of the final p-value of the corresponding r-value

In [ ]:
final_pval_list = []
for i in range(0,18+1):
    neuro_list = []
    for j in range(1,100+1):
        dat = getattr(mod,'dat{}'.format(j))
        neuro = dat.T.iloc[i,1]
        neuro_list.append(neuro)
    neuro_list = np.array(neuro_list)
    neuro_pval = np.unique(neuro_list, return_counts=True)[0]
    neuro_num = np.unique(neuro_list, return_counts=True)[1]
    final_pval = neuro_pval[np.argmax(neuro_num)]
    final_pval_list.append(final_pval)
final_pval_list = np.array(final_pval_list)

In [ ]:
dat_final = pd.DataFrame([x for x in zip(rval,final_pval_list)]).T
dat_final.columns = [i for i in receptor_name]
dat_final.index = ['r_val','p_val']
dat_final = np.round(dat_final,2)

save the above result as a csv file

In [ ]:
dat_final.to_csv('./fig3/permutation_result/final_dat.csv')

- this is final r-value and p-value

In [ ]:
dat_final

- import data_final and reorder receptors and transporters

In [ ]:
dat_final = pd.read_csv('./fig3/permutation_result/final_dat.csv',index_col=0)

In [ ]:
dat_final = dat_final.iloc[:,[8,9,10,16,0,1,2,3,4,5,6,13,18,17,14,11,12,7,15]]

### receptor barh plot

In [ ]:
colors = np.repeat(np.array(['blue','red','blue','red','blue','red','blue','red','blue']),[2,1,3,2,1,1,5,1,3],axis=0)

In [ ]:
dat_final_r = dat_final.iloc[:,::-1]

In [ ]:
fig = plt.figure(figsize=(10,25))
ax = fig.add_axes([0.1,0.1,0.8,0.8])
plt.barh(dat_final_r.columns, dat_final_r.iloc[0], height=0.2, color=colors[::-1],alpha=0.5)# plt.yticks(y, years)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# plt.gca().axes.get_yaxis().set_visible(False)
# plt.axis('off')

ax.set_xticks([-0.3, -0.2, -0.1, 0, 0.1, 0.2])
# ax.set_xticklabels(['', '', '', '', '', ''])

ax.tick_params(axis='x', direction='in',length=6)

# plt.savefig('./fig3/receptor_barh.png')

- draw a receptor atlas next to the neurotransmitter distribution barh plot

In [ ]:
from brainspace.datasets import load_group_fc, load_parcellation, load_conte69
from brainspace.gradient import GradientMaps
from brainspace.plotting import plot_hemispheres
from brainspace.utils.parcellation import map_to_labels

labeling = load_parcellation('schaefer', scale=200, join=True)
mask = labeling != 0

# schaefer200 기준으로 labeling과 mask를 생성

surf_lh, surf_rh = load_conte69()

- receptor/transceptor atlas

In [ ]:
for i in range(0,18+1):
    data = [None]*1
    data[0] = map_to_labels(receptor[:,i], labeling, mask=mask, fill=np.nan)
    
    name = dat.columns[i]

    plot_hemispheres(surf_lh, surf_rh, array_name=data, size=(1200,600), 
                 cmap='RdBu_r', color_bar=False, zoom=1.25, screenshot=True,
                 filename='C:/Users/chaehyeon/Dropbox/jupyter_notebook/camin2021/fig3/receptor_atlas/{}.png'.format(name))